# Simon Smith

## Overview

Simon Smith from curatorial team asked to to run a title comparison between NTLC title agains tiles in Mediaflex. 

### Objectives

1. Find any match title for NFSA main repo in Mediaflex against each NTLC title and provide a matching score to indicate how close the title match is.

2. Provide a Power BI report together with the analysis excel file for Simon to further study like which system holds the best quality of the video, whether the matched video are indeed the same.

## Solution Design





In [8]:
import pandas as pd

# Path to the Excel files
file_path_ntlc = '../data/NTLC/NTLC Collection Output April 2020.xlsx'
file_path_mdd = '../data/Mediaflex/MDD-MoveList-Form-20240717.xlsx'
# file_path_mdd = '../data/Mediaflex/MDD-MoveList-All-20240717.xlsx'


# Read the Excel files
df_ntlc = pd.read_excel(file_path_ntlc)
df_mdd = pd.read_excel(file_path_mdd, sheet_name='Export Worksheet')

# Display the first few rows of the DataFrame
# print(df_ntlc.head())
# print(df_mdd.head())


c:\Users\User\.conda\envs\nfsa-env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


   VERSION_ID  TITLE_ID_FK              MOVIETITLE VERSIONMEDIUM  VERSIONNO  \
0       46705        46671           THE LAST WAVE          Film     400323   
1       46706        46671           THE LAST WAVE          Film     496831   
2       47156        47155  PICNIC AT HANGING ROCK          Film          3   
3       47235        47234               DILLINGER          Film       8254   
4       47287        47155  PICNIC AT HANGING ROCK          Film     629872   

             PROGRAMTITLE PROGRAMMEDIUM PROGRAMSUBMEDIUM  PROGRAMYEAR  \
0           THE LAST WAVE  Moving Image             Film            0   
1           THE LAST WAVE  Moving Image             Film            0   
2  PICNIC AT HANGING ROCK  Moving Image             Film            0   
3               DILLINGER  Moving Image             Film            0   
4  PICNIC AT HANGING ROCK  Moving Image             Film            0   

  CONCATENATED_COUNTRIES CONCATENATED_LANGUAGES CONCATENATED_FORMS  
0              Au

In [13]:
import pandas as pd
from fuzzywuzzy import process

# Load the Excel files
file_path_ntlc = '../data/NTLC/NTLC Collection Output April 2020.xlsx'
file_path_mdd = '../data/Mediaflex/MDD-MoveList-Form-20240717v1.xlsx'

df_ntlc = pd.read_excel(file_path_ntlc)
df_mdd = pd.read_excel(file_path_mdd, sheet_name='Export Worksheet')

# Convert FTITLE and MOVIETITLE columns to strings
df_ntlc['FTITLE'] = df_ntlc['FTITLE'].astype(str)
df_mdd['MOVIETITLE'] = df_mdd['MOVIETITLE'].astype(str)

# Function to get the best match title and score
def get_best_match_title(row, choices):
    title = row['FTITLE']
    best_match, score = process.extractOne(title, choices)
    return pd.Series([best_match, score])

# Find the best match title and score for each record in df_ntlc
choices = df_mdd['MOVIETITLE'].tolist()
df_ntlc[['MOVIETITLE', 'FuzzyMatchingScore']] = df_ntlc.apply(get_best_match_title, choices=choices, axis=1)

# Merge df_ntlc with the relevant columns from df_mdd
df_mdd_subset = df_mdd[['MOVIETITLE', 'VERSION_ID', 'TITLE_ID_FK', 'VERSIONNO', 'PROGRAMTITLE', 'PROGRAMYEAR', 'CONCATENATED_COUNTRIES', 'CONCATENATED_LANGUAGES', 'CONCATENATED_FORMS']]
result = pd.merge(df_ntlc, df_mdd_subset, on='MOVIETITLE', how='left')

# Save the result to a new Excel file
output_file_path = '../data/output/matching_results.xlsx'
result.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


Results saved to ../data/output/matching_results.xlsx


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def calculate_similarity_score(title1, title2):
    return fuzz.ratio(title1, title2) / 100.0

# Create a function to find the best match and score for each title
def find_best_matches(system1_titles, system2_titles):
    matches = []
    for title1 in system1_titles:
        best_match = None
        highest_score = 0
        for title2 in system2_titles:
            score = calculate_similarity_score(title1, title2)
            if score > highest_score:
                highest_score = score
                best_match = title2
        matches.append((title1, best_match, highest_score))
    return matches

# Get the matches and scores
matches = find_best_matches(system1_titles, system2_titles)

# Print the results
for title1, best_match, score in matches:
    print(f"'{title1}' best matches with '{best_match}' with a score of {score:.2f}")


In [21]:
import pandas as pd
from fuzzywuzzy import fuzz

# Load the Excel files
file_path_ntlc = '../data/NTLC/NTLC Collection Output April 2020v1.xlsx'
file_path_mdd = '../data/Mediaflex/MDD-MoveList-Form-20240717.xlsx'

df_ntlc = pd.read_excel(file_path_ntlc)
df_mdd = pd.read_excel(file_path_mdd, sheet_name='Export Worksheet')

# Convert FTITLE and MOVIETITLE columns to strings
df_ntlc['FTITLE'] = df_ntlc['FTITLE'].astype(str)
df_mdd['MOVIETITLE'] = df_mdd['MOVIETITLE'].astype(str)

# Function to get the best match title and score using fuzz.ratio
def get_best_match_title(row, choices):
    title = row['FTITLE'].lower()
    best_match, score = None, 0
    for choice in choices:
        current_score = fuzz.ratio(title, choice.lower())
        #  current_score = fuzz.token_set_ratio(title, choice.lower())
        #  current_score = fuzz.token_set_ratio(title, choice.lower())
        if current_score > score:
            best_match, score = choice, current_score
    return pd.Series([best_match, score])

# Find the best match title and score for each record in df_ntlc
choices = df_mdd['MOVIETITLE'].tolist()
df_ntlc[['MOVIETITLE', 'FuzzyMatchingScore']] = df_ntlc.apply(get_best_match_title, choices=choices, axis=1)

# Merge df_ntlc with the relevant columns from df_mdd
df_mdd_subset = df_mdd[['MOVIETITLE', 'VERSION_ID', 'TITLE_ID_FK', 'VERSIONNO', 'PROGRAMTITLE', 'PROGRAMYEAR', 'CONCATENATED_COUNTRIES', 'CONCATENATED_LANGUAGES', 'CONCATENATED_FORMS']]
result = pd.merge(df_ntlc, df_mdd_subset, on='MOVIETITLE', how='left')

# Save the result to a new Excel file
output_file_path = '../data/output/matching_results.xlsx'
result.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


c:\Users\User\.conda\envs\nfsa-env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Results saved to ../data/output/matching_results.xlsx


In [23]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed

# Load the Excel files
file_path_ntlc = '../data/NTLC/NTLC Collection Output April 2020v1.xlsx'
file_path_mdd = '../data/Mediaflex/MDD-MoveList-Form-20240717.xlsx'

df_ntlc = pd.read_excel(file_path_ntlc)
df_mdd = pd.read_excel(file_path_mdd, sheet_name='Export Worksheet')

# Convert FTITLE and MOVIETITLE columns to strings
df_ntlc['FTITLE'] = df_ntlc['FTITLE'].astype(str)
df_mdd['MOVIETITLE'] = df_mdd['MOVIETITLE'].astype(str)

# Function to get the best match title and score using fuzz.ratio
def get_best_match_title(row_title, choices):
    title = row_title.lower()
    scores = np.array([fuzz.ratio(title, choice.lower()) for choice in choices])
    best_match_index = scores.argmax()
    return choices[best_match_index], scores[best_match_index]

# Convert choices to a list
choices = df_mdd['MOVIETITLE'].tolist()

# Apply get_best_match_title in parallel
results = Parallel(n_jobs=-1, backend="multiprocessing")(delayed(get_best_match_title)(title, choices) for title in df_ntlc['FTITLE'])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['MOVIETITLE', 'FuzzyMatchingScore'])

# Add results to df_ntlc
df_ntlc[['MOVIETITLE', 'FuzzyMatchingScore']] = results_df

# Merge df_ntlc with the relevant columns from df_mdd
df_mdd_subset = df_mdd[['MOVIETITLE', 'VERSION_ID', 'TITLE_ID_FK', 'VERSIONNO', 'PROGRAMTITLE', 'PROGRAMYEAR', 'CONCATENATED_COUNTRIES', 'CONCATENATED_LANGUAGES', 'CONCATENATED_FORMS']]
result = pd.merge(df_ntlc, df_mdd_subset, on='MOVIETITLE', how='left')

# Save the result to a new Excel file
output_file_path = '../data/output/matching_results.xlsx'
result.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


c:\Users\User\.conda\envs\nfsa-env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


KeyboardInterrupt: 

In [28]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import time

# Load the Excel files
file_path_ntlc = '../data/NTLC/NTLC Collection Output April 2020.xlsx'
file_path_mdd = '../data/Mediaflex/MDD-MoveList-Form-20240717.xlsx'

df_ntlc = pd.read_excel(file_path_ntlc)
df_mdd = pd.read_excel(file_path_mdd, sheet_name='Export Worksheet')

# Convert FTITLE and MOVIETITLE columns to strings
df_ntlc['FTITLE'] = df_ntlc['FTITLE'].astype(str)
df_mdd['MOVIETITLE'] = df_mdd['MOVIETITLE'].astype(str)

# Function to get the best match title and score using fuzz.ratio
def get_best_match_title(row_title, choices):
    title = row_title.lower()
    scores = np.array([fuzz.ratio(title, choice.lower()) for choice in choices])
    best_match_index = scores.argmax()
    return choices[best_match_index], scores[best_match_index]

# Convert choices to a list
choices = df_mdd['MOVIETITLE'].tolist()

# Measure start time
start_time = time.time()

# Apply get_best_match_title in parallel with progress bar
results = Parallel(n_jobs=12, backend="multiprocessing")(
    delayed(get_best_match_title)(title, choices) for title in tqdm(df_ntlc['FTITLE'], desc="Processing", unit="record")
)

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['MOVIETITLE', 'FuzzyMatchingScore'])

# Add results to df_ntlc
df_ntlc[['MOVIETITLE', 'FuzzyMatchingScore']] = results_df

# Merge df_ntlc with the relevant columns from df_mdd
df_mdd_subset = df_mdd[['MOVIETITLE', 'VERSION_ID', 'TITLE_ID_FK', 'VERSIONNO', 'PROGRAMTITLE', 'PROGRAMYEAR', 'CONCATENATED_COUNTRIES', 'CONCATENATED_LANGUAGES', 'CONCATENATED_FORMS']]
result = pd.merge(df_ntlc, df_mdd_subset, on='MOVIETITLE', how='left')

# Save the result to a new Excel file
output_file_path = '../data/output/matching_results.xlsx'
result.to_excel(output_file_path, index=False)

# Measure end time
end_time = time.time()

print(f"Results saved to {output_file_path}")
print(f"Processing time: {end_time - start_time} seconds")


c:\Users\User\.conda\envs\nfsa-env\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


KeyboardInterrupt: 